In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV

In [3]:
bank = pd.read_csv('bank/bank-full.csv', sep=';')

bank = bank.replace('yes', 1).replace('no', 0)

months = pd.DataFrame({
    'month': bank.month.sort_values().unique(),
    'month_no': [4, 8, 12, 2, 1, 7, 6, 3, 5, 11, 10, 9]
})
bank = bank.merge(months).drop('month', axis=1)

bank = bank.merge(pd.DataFrame({
    'education': bank.education.unique(),
    'edu': [1, 2, 3, np.nan]
})).drop('education', axis=1)

bank = bank.merge(pd.DataFrame({
    'poutcome': bank.poutcome.sort_values().unique(),
    'prev_out': [-1, np.nan, 1, 0]
})).drop('poutcome', axis=1)

bank = bank.drop('job', axis=1)

bank_dummies = pd.get_dummies(bank)
bank_dummies.sample(10)

bank_dummies = bank_dummies.dropna()

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    bank_dummies.drop('y', axis=1), # X
    bank_dummies.y,  # y
    test_size=0.2, # porcentaje que será prueba
    random_state=42) # para fijar el aleatorio (reproducibilidad)

In [12]:
grid_parametros = {
    'criterion':['gini','entropy'],
    'max_depth':[i for i in range (1,100)],
    'min_samples_split':[i for i in range(100,500,50)],
    
}
dtc = DecisionTreeClassifier()
rf = DecisionTreeClassifier()

rf_cv = GridSearchCV(rf, grid_parametros,
                     n_jobs=-1, scoring='f1')

In [13]:
dtc.fit(X_train, y_train)
rf_cv.fit(X_train, y_train)

/Users/rvbi/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/rvbi/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/rvbi/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/rvbi/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/rvbi/anaconda3/lib/python3.7/site-packages/sklearn/metric

GridSearchCV(cv=None, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'criterion': ['gini', 'entropy'], 'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 6... 91, 92, 93, 94, 95, 96, 97, 98, 99], 'min_samples_split': [100, 150, 200, 250, 300, 350, 400, 450]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1', verbose=0)

In [14]:
rf_cv.best_params_

{'criterion': 'entropy', 'max_depth': 17, 'min_samples_split': 250}

In [15]:
rf_cv.best_score_

0.5360508449951163

In [16]:
f1_score(y_train, rf_cv.predict(X_train))

0.5983035443804907

In [17]:
f1_score(y_test, rf_cv.predict(X_test))

0.5412899336949971

In [18]:
f1_score(y_test, dtc.predict(X_test)) 

0.49890590809628005